# Data extraction

In [1]:
import os
import open3d as o3d
import octomap
import torch
import numpy as np
from symlink import symbolic_dir
from utils_o3d import Get_Pointcloud, Get_RGBD, scale_and_translate, Get_voxpoints

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#print("Ingresa la dirección a la carpeta contenedora del banco de datos")
#direccion = input("Cúal es la dirección a la carpeta contenedora?:") # Dataset acces
direccion = 'objetos'
x = os.listdir(direccion)
carpeta = input("A que carpeta quieres acceder?: ") #object folder
direccion_disco = '/mnt/6C24E28478939C77/Saulo/ProyectoPHD/'
dir_carpeta = direccion_disco + direccion #+ "/" + carpeta
if os.path.lexists(dir_carpeta + "/meshes/texture.png") == False:
    symbolic_dir(dir_carpeta)
    RGB = "/RGB"
    Depth = "/Depth"
    Point_cloud = "/Point_cloud"
    Octree = "/Octree"
    os.mkdir(dir_carpeta + RGB)
    os.mkdir(dir_carpeta + Depth)
    os.mkdir(dir_carpeta + Point_cloud)
    os.mkdir(dir_carpeta + Octree)
visualizar = input("Quieres visualizar el objeto? (presiona S en caso de sí):") 
if visualizar == 'S'  or  visualizar == 's' :
    mesh = o3d.io.read_triangle_mesh(dir_carpeta + '/meshes/model.obj', True) 
    orig = o3d.geometry.TriangleMesh.create_coordinate_frame()
    o3d.visualization.draw_geometries([mesh])

img_W = 1920
img_H = 1080


#Cargamos malla
mesh = o3d.io.read_triangle_mesh(dir_carpeta + '/meshes/model.obj', True)
material = o3d.visualization.rendering.MaterialRecord() # Create material
material.albedo_img = o3d.io.read_image( dir_carpeta + '/meshes/texture.png') # Add texture
mesh = scale_and_translate(mesh, scale_factor=0.39)

# Raycasting
mesh1 = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
scene = o3d.t.geometry.RaycastingScene()
scene.add_triangles(mesh1)
# render for RGBD images
render = o3d.visualization.rendering.OffscreenRenderer(width=img_W, height=img_H) #Linux only
render.scene.add_geometry('mesh', mesh, material)
#Camera vectors setup
cent = [0,0,.2]#mesh.get_center()
up = [0, 1, 0]
poses = np.load("esferas_v.npz")
fov = 45

for i in range(0,len(poses["r"])):

    for j in ["r","r1","r2"]:  
        eye = poses[j][i]
        # RGBD and pointcloud extraction
        Get_Pointcloud(scene, fov, cent, eye, up, img_W, img_H, dir_carpeta, j+'_'+str(i))
        #Get_RGBD(render,  fov, cent, eye, up, dir_carpeta, j+'_'+str(i))
        
    


[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0xb184950 (threading is enabled)
EGL(1.5)
OpenGL(4.1)


# For multiple objects

In [2]:
############# P A R A M E T R O S #####################################################################
img_W = 1920
img_H = 1080
#Camera vectors setup
cent = [0,0,.2]#mesh.get_center()
up = [0, 1, 0]
poses = np.load("esferas_v.npz")
fov = 45
direccion = "/mnt/6C24E28478939C77/Saulo/ProyectoPHD/"#Direccion a carpeta contenedora
objeto = "objects"
dir_carpeta = direccion + objeto + "/"
listado = os.listdir(dir_carpeta)
#####################################################################################################

for l in range (0,len(listado)):
    puntos = Get_voxpoints()
    direction = dir_carpeta + listado[l]
    #Cargamos malla
    mesh = o3d.io.read_triangle_mesh(direction + '/meshes/model.obj', True)
    material = o3d.visualization.rendering.MaterialRecord() # Create material
    material.albedo_img = o3d.io.read_image( direction + '/meshes/texture.png') # Add texture
    mesh = scale_and_translate(mesh, scale_factor=0.39)

    # Raycasting
    mesh1 = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
    scene = o3d.t.geometry.RaycastingScene()
    scene.add_triangles(mesh1)
    for i in range(0,len(poses["r"])):
        for j in ["r","r1","r2"]:  
            eye = poses[j][i]
            # RGBD and pointcloud extraction
            Get_Pointcloud(scene, fov, cent, eye, up, img_W, img_H, direction, j+'_'+str(i))
            #Get_RGBD(render,  fov, cent, eye, up, dir_carpeta, j+'_'+str(i))

    del mesh
    del material
    del mesh1
    del scene